# 计算图

计算图是用来描述运算的有向无环图。计算图有两个主要元素：结点(Node)和边(Edge)，结点表示数据，如向量，矩阵，张量；边表示运算，如加减乘除卷积等。  

用计算图表示 $y=(x+w)\times(w+1),\quad a=x+w,b=w+1,y=a*b$

<img src="img/计算图.png" width="300">

## 计算图与梯度求导

根据复合函数链式求导法则，体现到计算图中，就是根节点 $y$ 到叶子节点 $w$ 有两条路径 $y\rightarrow  a\rightarrow w$ 和 $y\rightarrow b\rightarrow w$。根节点依次对每条路径的孩子节点求导，一直到叶子节点 $w$，最后把每条路径的导数相加即可。

$$\begin{aligned} 
\frac{\partial y}{\partial w} &=\frac{\partial y}{\partial a} \frac{\partial a}{\partial w}+\frac{\partial y}{\partial b} \frac{\partial b}{\partial w} \\ 
&=b \times 1+a \times 1 \\ 
&=(w+1)+(x+w) \\ 
&=2  w+x+1 
\end{aligned}$$

<img src="img/计算图与梯度求导.png" width="300">

In [1]:
import torch
import warnings
warnings.filterwarnings('ignore')

# 令w=1，x=2
w = torch.tensor([1.],requires_grad=True)
x = torch.tensor([2.],requires_grad=True)
a = x + w
b = w + 1
y = a * b

# 反向传播
y.backward()
print(w.grad)    

tensor([5.])


`Tensor`中有一个属性`is_leaf`标记是否为叶子节点，在上面的例子中，$x$ 和 $w$ 是叶子节点，其他所有节点都依赖于叶子节点。叶子节点的概念主要是为了节省内存，在计算图中的一轮反向传播结束之后，非叶子节点的梯度是会被释放的。

In [2]:
# 查看叶子节点
print(f'is leaf:\n{w.is_leaf},{x.is_leaf},{a.is_leaf},{b.is_leaf},{y.is_leaf}')

is leaf:
True,True,False,False,False


In [3]:
# 查看梯度
print(f'gradient:\n{w.grad},{x.grad},{a.grad},{b.grad},{y.grad}')

gradient:
tensor([5.]),tensor([2.]),None,None,None


如果想要保存非叶子节点的梯度，可以在执行反向传播前调用`retain_grad()`方法

In [4]:
w = torch.tensor([1.],requires_grad=True)
x = torch.tensor([2.],requires_grad=True)
a = x + w
a.retain_grad()    # 保留a的梯度
b = w + 1
y = a * b
y.backward()
print(f'gradient:\n{w.grad},{x.grad},{a.grad},{b.grad},{y.grad}')

gradient:
tensor([5.]),tensor([2.]),tensor([2.]),None,None


`Tensor`中的`grad_fn`属性记录的是创建该张量时所用的方法 (函数)。而在反向传播求导梯度时需要用到该属性。

In [5]:
print(f'grad_fn:\n{y.grad_fn}\n{a.grad_fn}\n{b.grad_fn}\n{w.grad_fn}\n{x.grad_fn}')

grad_fn:
None
None


从运行结果可以看出，$y$ 是通过乘法计算得到的，$a$ 和 $b$ 是通过加法计算得到的。

## 叶子节点和tensor的requires_grad参数

tensor可细分为两类：叶子节点(leaf node)和非叶子节点。  

使用`backward()`函数反向传播计算tensor的梯度时，并不计算所有tensor的梯度，而是只计算满足这几个条件的tensor的梯度：  
1. 类型为叶子节点
2. `requires_grad=True`
3. 依赖该tensor的所有tensor的`requires_grad=True`。  

叶子节点可以理解成**不依赖其他tensor的tensor**。在pytorch中，神经网络层中的权值w的tensor均为叶子节点；自己定义的tensor例如`a=torch.tensor([1.0])`是叶子节点。

In [6]:
a=torch.tensor([1.0])
print(a.is_leaf)

b=a+1
print(b.is_leaf)

True
True


可以看出b竟然也是叶子节点。这件事可以这样理解，单纯从数值关系上b=a+1，b确实依赖a。但是从pytorch的看来，一切是为了反向求导，a的`requires_grad`属性为False，其不要求获得梯度，那么a这个tensor在反向传播时其实是“无意义”的，可认为是游离在计算图之外的，故b仍然为叶子节点，如下图  

<img src="img/叶子节点1.jpg" width="500">  

对于需要求导的tensor，其requires_grad属性必须为True，例如对于下图中最上面的叶子节点，pytorch不会自动计算其导数。  

<img src="img/叶子节点2.jpg" width="500">  

自己定义的tensor的`requires_grad`属性默认为False，神经网络层中的权值w的tensor的`requires_grad`属性默认为True。  

如果自行定义了一个tensor并将其`requires_grad`设置为`True`，该tensor是叶子节点，且依赖该tensor的其他tensor是非叶子节点（非叶子节点不会自动求导），其`requires_grad`自动设置为`True`，这样便形成了一条从叶节点到loss节点的求导的“通路”。  

如果需要使得某一个节点成为叶子节点，只需使用`detach()`即可将它从创建它的计算图中分离开来。

# 动态图

PyTorch 采用的是动态图机制 (Dynamic Computational Graph)，而 Tensorflow 采用的是静态图机制 (Static Computational Graph)。  

动态图是**运算和搭建同时进行**，也就是可以先计算前面的节点的值，再根据这些值搭建后面的计算图。优点是**灵活，易调节，易调试**。PyTorch 里的很多写法跟其他 Python 库的代码的使用方法是完全一致的，没有任何额外的学习成本。  

静态图是**先搭建图，然后再输入数据进行运算**。优点是**高效**，因为静态计算是通过先定义后运行的方式，之后再次运行的时候就不再需要重新构建计算图，所以速度会比动态图更快。但是**不灵活**。TensorFlow 每次运行的时候图都是一样的，是不能够改变的，所以不能直接使用 Python 的 while 循环语句，需要使用辅助函数 tf.while_loop 写成 TensorFlow 内部的形式。

静态图就是将计算图的过程搭建完成，比如下图，然后将数据输入，一步步按照下面的方式执行，也就是tensor像流动一样完成流程，所以称为Tensorflow。简单来说，就是先搭建图，然后运算。

<img src="img/tensors_flowing.gif">

而在pytorch中，首先创建Tensor，下面例子中的图只出现四个张量，然后进行一步步运算，整个图不断完善，直到最后一步，形成这样的一个流程图。  

<img src="img/动态图.jpg" width="600">